In [2]:
import geopandas
import fiona
import pandas as pd
import numpy as np

In [3]:
file=geopandas.read_file("klinj_suojatie.gpkg")
df=pd.DataFrame(file)
df.geometry=df.geometry.astype("str")
df["geometry"]=df["geometry"].str.replace("LINESTRING ","").str.replace(",","").str.replace("(","").str.replace(")","")
print(df.loc[0,"geometry"])
print(df.loc[1,"geometry"])
print(df.loc[2,"geometry"])
print(df.loc[3,"geometry"])
# insconsistent order of minx and maxx
# inconsistent order of miny and maxy

25497295.9682035 6672929.268 25497296.3567031 6672912.646
25497172.2023273 6673374.486 25497179.4363201 6673370.764
25497334.3101652 6672931.001 25497334.2901652 6672914.92
25497361.1891383 6672770.811 25497354.3851451 6672776.495


In [4]:
df[["min_x","max_y","max_x","min_y","a","b"]]=df["geometry"].str.split(" ",expand=True)
df=df.drop(columns=["a","b","geometry"])
df["id"]=df["gml_id"].str.replace("klinj_suojatie.","")

In [5]:
df["min_x"]=df["min_x"].astype(float)
df["min_y"]=df["min_y"].astype(float)
df["max_x"]=df["max_x"].astype(float)
df["max_y"]=df["max_y"].astype(float)
df


,gml_id,id,vaylatyyppi,luokka,featurecode,html_color,min_x,max_y,max_x,min_y
0,klinj_suojatie.16704,16704,Suojatie,Suojatiet,12,400040,2.549730e+07,6672929.268,2.549730e+07,6672912.646
1,klinj_suojatie.16713,16713,Suojatie,Suojatiet,12,400040,2.549717e+07,6673374.486,2.549718e+07,6673370.764
2,klinj_suojatie.16741,16741,Suojatie,Suojatiet,12,400040,2.549733e+07,6672931.001,2.549733e+07,6672914.920
3,klinj_suojatie.16908,16908,Suojatie,Suojatiet,12,400040,2.549736e+07,6672770.811,2.549735e+07,6672776.495
4,klinj_suojatie.16910,16910,Suojatie,Suojatiet,12,400040,2.549735e+07,6672758.490,2.549734e+07,6672759.135
...,...,...,...,...,...,...,...,...,...,...
1388,klinj_suojatie.96058,96058,Suojatie,Suojatiet,12,400040,2.549739e+07,6672374.249,2.549738e+07,6672370.090
1389,klinj_suojatie.96553,96553,Suojatie,Suojatiet,12,400040,2.549660e+07,6673253.985,2.549660e+07,6673246.648
1390,klinj_suojatie.96573,96573,Suojatie,Suojatiet,12,400040,2.549641e+07,6673288.712,2.549641e+07,6673292.230
1391,klinj_suojatie.96574,96574,Suojatie,Suojatiet,12,400040,2.549641e+07,6673290.074,2.549641e+07,6673290.186


In [6]:
# Fixing inconsistencies 
df.min_x, df.max_x = np.where(df.min_x > df.max_x, [df.max_x, df.min_x], [df.min_x, df.max_x])
df.min_y, df.max_y = np.where(df.min_y > df.max_y, [df.max_y, df.min_y], [df.min_y, df.max_y])

In [8]:
from owslib.wms import WebMapService
import os

resolution = 100 

layer = "Ortoilmakuva_2019_5cm"
#layer = "Vaaravariortoilmakuva_2019_20cm"

folder_name ="/Users/alexanderjung/PedestrianCrossingImg/resolution_"+str(resolution)+"_layer_"+layer+"/" 


if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    
#os.mkdir("images")
#os.mkdir("images/phase2_crossroad_images_1")
#os.mkdir("images/resolution"+str(resolution)+"/")
wms = WebMapService('https://kartta.hel.fi/ws/geoserver/avoindata/wms', version='1.3.0')

print(wms.contents)


for i in range(0,len(df)):
#for i in range(0,4):
    minx = df.loc[i,"min_x"] 
    miny = df.loc[i,"min_y"]
    maxx = df.loc[i,"max_x"]
    maxy = df.loc[i,"max_y"]
    deltax = (maxx-minx)
    deltay = (maxy-miny)
    roi = (minx-deltax,miny-deltay,maxx+deltax,maxy+deltay)
 
    img=wms.getmap(layers=[layer],srs="EPSG:3879",bbox=roi,size=(resolution,resolution),format='image/png',transparent=True)
    filename =folder_name + str(df.loc[i,"id"])+".png"
    #filename  = "pedestrian_crossing" + str(i)+".png"
    out = open(filename, 'wb')
    out.write(img.read())
    out.close()


OrderedDict([('avoindata:Kantakartta', <owslib.map.wms130.ContentMetadata object at 0x1268b7a10>), ('avoindata:Kantakartta_harmaa', <owslib.map.wms130.ContentMetadata object at 0x1268db310>), ('avoindata:Kantakartta_varillinen', <owslib.map.wms130.ContentMetadata object at 0x1268dbb90>), ('avoindata:Kantakartta_rajat', <owslib.map.wms130.ContentMetadata object at 0x1268db490>), ('avoindata:Kantakartta_valkoinen', <owslib.map.wms130.ContentMetadata object at 0x1268e1c50>), ('avoindata:Kantakartan_maastotiedot', <owslib.map.wms130.ContentMetadata object at 0x1268e64d0>), ('avoindata:Kantakartta_pohja', <owslib.map.wms130.ContentMetadata object at 0x1268d6ad0>), ('avoindata:Kantakartta_paakohteet', <owslib.map.wms130.ContentMetadata object at 0x1268e6590>), ('avoindata:Karttasarja', <owslib.map.wms130.ContentMetadata object at 0x1268eddd0>), ('avoindata:Karttasarja_harmaa', <owslib.map.wms130.ContentMetadata object at 0x1268e1550>), ('avoindata:Kiinteistokartta', <owslib.map.wms130.Conten